### Importing libraries

In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
import geojson
import json
pd.set_option('display.min_rows', 1000)

##### Loading the geojson file into jupyter

Before manipulating our temperature data, I thought it would be a good first step to download the geojson file that will be used to display the country polygons on the web page. Given the initial geojson does not contain the temperature values we'll need for our chloropleth map, these will need to be inserted manually using python. First, I downloaded the geojson file from: https://gist.github.com/phil-pedruco/10447085#file-countries-geo-json using the following code:

In [2]:
with open('countries.geo.json') as f:
    gj = geojson.load(f)
features = gj['features']

##### Read in the Excel file
Next, we can download the temperature predictions from the Kaggle World Bank repository: https://www.kaggle.com/theworldbank/world-bank-climate-change-data. We can use pandas to read in the projections and rename a few columns to make it easier for data manipulation:

In [3]:
climate = pd.read_excel('climate_change_projections.xlsx')
climate = climate.rename(columns= {"ISO_3DIGIT": "Country", "Value": "Change_c", "Timeperiod_Variable_month_scenario_ensemble": "Timeperiod"})

### Tidying the projection data
With our projection data successfully read into our jupyter notebook, we can now do some tidying. The end goal is to create final json files that will include the temperature projections for each country, split out by time period and scenario. First, we'll use `groupby` to group our projections for each country, still split out by different time periods, percentiles, and scenarios.

In [73]:
climate_grp = climate.groupby(['Timeperiod', 'Country']).sum()['Change_c'].reset_index().sort_values(['Timeperiod', 'Change_c'])

In [75]:
climate_grp.head(5)

,Timeperiod,Country,Change_c
115,2020-2040_tas_01__a2_10th,LAO,-0.171904
84,2020-2040_tas_01__a2_10th,GRL,-0.091518
39,2020-2040_tas_01__a2_10th,CIV,-0.080205
225,2020-2040_tas_01__a2_10th,VNM,-0.072714
75,2020-2040_tas_01__a2_10th,GHA,-0.065681


As we can see above, we are able to get the projected change in degrees C for every country, time period, and scenario, however, the `Timeperiod` column has all of these factors grouped into one variable. In order for us to split this data further, we'll have to split up this `Timeperiod` value into different columns. We can do this with the following syntax below:

In [76]:
climate_grp[['Date_range', 'Model', 'Month', 'Blank', 'Type', 'Pctile']] = climate_grp['Timeperiod'].str.split('_',expand=True)

In [77]:
climate_grp.head(5)

,Timeperiod,Country,Change_c,Date_range,Model,Month,Blank,Type,Pctile
115,2020-2040_tas_01__a2_10th,LAO,-0.171904,2020-2040,tas,01,,a2,10th
84,2020-2040_tas_01__a2_10th,GRL,-0.091518,2020-2040,tas,01,,a2,10th
39,2020-2040_tas_01__a2_10th,CIV,-0.080205,2020-2040,tas,01,,a2,10th
225,2020-2040_tas_01__a2_10th,VNM,-0.072714,2020-2040,tas,01,,a2,10th
75,2020-2040_tas_01__a2_10th,GHA,-0.065681,2020-2040,tas,01,,a2,10th


Now, we have all of the important factors split into separate columns in our pandas dataframe. We can continue to group our data accordingly to first group by country, date range, percentile and scenario -- we'll be taking the mean temperature in degrees C across the twelve months for each time period and rolling that up for each country.

In [78]:
climate_grp = climate_grp.groupby(['Country', 'Date_range', 'Pctile', 'Type']).mean().reset_index()

In [83]:
climate_grp.head(24)

,Country,Date_range,Pctile,Type,Change_c
0,ABW,2020-2040,10th,a2,0.668032
1,ABW,2020-2040,10th,b1,0.569697
2,ABW,2020-2040,90th,a2,1.181766
3,ABW,2020-2040,90th,b1,1.184365
4,ABW,2020-2040,median,a2,0.972389
5,ABW,2020-2040,median,b1,0.960821
6,ABW,2040-2060,10th,a2,1.061561
7,ABW,2040-2060,10th,b1,0.853551
8,ABW,2040-2060,90th,a2,1.828642
9,ABW,2040-2060,90th,b1,1.486765


Our data manipulation yields the above for every country in our dataset. The example shown is for the country of Aruba (ABW), where we can see mean temperature projections for each time period split out by the 10th, 90th and median outputs of the model, and also split out between our two scenarios (a2 and b1).

Next, we'll convert the temperature projections from degrees Celsius to degrees Fahrenheit, to make our visualizations more digestable for folks in the US.

In [84]:
climate_grp['Change_f'] = climate_grp['Change_c'] * 1.8

After this, I separated the projections into four different dataframes:
- `first20`: Date range from 2020 - 2040
- `second20`: Date range from 2040 - 2060
- `third20`: Date range from 2060 - 2080
- `fourth20`: Date range from 2080 - 2100

In [8]:
first20 = climate_grp[climate_grp['Date_range'] == '2020-2040']
second20 = climate_grp[climate_grp['Date_range'] == '2040-2060']
third20 = climate_grp[climate_grp['Date_range'] == '2060-2080']
fourth20 = climate_grp[climate_grp['Date_range'] == '2080-2100']

#### Split data based on models and projected percentiles

Now, with our data split out into separate dataframes per time period, the final step will be to split the four dataframes out into eight dataframes to accommodate the projections for the two different scenarios. Below, I used the `loc` method to filter the projections accordingly and saved them to separate dataframes:

In [ ]:
first20_med_a2 = first20.loc[(first20['Pctile'] == 'median') & (first20['Type'] == 'a2')].sort_values('Change_f')
first20_med_b1 = first20.loc[(first20['Pctile'] == 'median') & (first20['Type'] == 'b1')].sort_values('Change_f')
second20_med_a2 = second20.loc[(second20['Pctile'] == 'median') & (second20['Type'] == 'a2')].sort_values('Change_f')
second20_med_b1 = second20.loc[(second20['Pctile'] == 'median') & (second20['Type'] == 'b1')].sort_values('Change_f')
third20_med_a2 = third20.loc[(third20['Pctile'] == 'median') & (third20['Type'] == 'a2')].sort_values('Change_f')
third20_med_b1 = third20.loc[(third20['Pctile'] == 'median') & (third20['Type'] == 'b1')].sort_values('Change_f')
fourth20_med_a2 = fourth20.loc[(fourth20['Pctile'] == 'median') & (fourth20['Type'] == 'a2')].sort_values('Change_f')
fourth20_med_b1 = fourth20.loc[(fourth20['Pctile'] == 'median') & (fourth20['Type'] == 'b1')].sort_values('Change_f')

For my final projection visualizations, I thought it would be best to take the **median** projections for each scenario and time period. However, below is the code to isolate out the 10th and 90th percentile projections for each of these scenarios and time periods if it's helpful for future work.

In [85]:
## separating the remaining scenarios and projections into additional dataframes

# first20_ten_a2 = first20.loc[(first20['Pctile'] == '10th') & (first20['Type'] == 'a2')].sort_values('Change_f')
# first20_ninety_a2 = first20.loc[(first20['Pctile'] == '90th') & (first20['Type'] == 'a2')].sort_values('Change_f')
# first20_ten_b1 = first20.loc[(first20['Pctile'] == '10th') & (first20['Type'] == 'b1')].sort_values('Change_f')
# first20_ninety_b1 = first20.loc[(first20['Pctile'] == '90th') & (first20['Type'] == 'b1')].sort_values('Change_f')
# second20_ten_a2 = second20.loc[(second20['Pctile'] == '10th') & (second20['Type'] == 'a2')].sort_values('Change_f')
# second20_ninety_a2 = second20.loc[(second20['Pctile'] == '90th') & (second20['Type'] == 'a2')].sort_values('Change_f')
# second20_ten_b1 = second20.loc[(second20['Pctile'] == '10th') & (second20['Type'] == 'b1')].sort_values('Change_f')
# second20_ninety_b1 = second20.loc[(second20['Pctile'] == '90th') & (second20['Type'] == 'b1')].sort_values('Change_f')
# third20_ten_a2 = third20.loc[(third20['Pctile'] == '10th') & (third20['Type'] == 'a2')].sort_values('Change_f')
# third20_ninety_a2 = third20.loc[(third20['Pctile'] == '90th') & (third20['Type'] == 'a2')].sort_values('Change_f')
# third20_ten_b1 = third20.loc[(third20['Pctile'] == '10th') & (third20['Type'] == 'b1')].sort_values('Change_f')
# third20_ninety_b1 = third20.loc[(third20['Pctile'] == '90th') & (third20['Type'] == 'b1')].sort_values('Change_f')
# fourth20_ten_a2 = fourth20.loc[(fourth20['Pctile'] == '10th') & (fourth20['Type'] == 'a2')].sort_values('Change_f')
# fourth20_ninety_a2 = fourth20.loc[(fourth20['Pctile'] == '90th') & (fourth20['Type'] == 'a2')].sort_values('Change_f')
# fourth20_ten_b1 = fourth20.loc[(fourth20['Pctile'] == '10th') & (fourth20['Type'] == 'b1')].sort_values('Change_f')
# fourth20_ninety_b1 = fourth20.loc[(fourth20['Pctile'] == '90th') & (fourth20['Type'] == 'b1')].sort_values('Change_f')

### Merging the projection data into the existing geojson file and creating separate geojsons

Finally, with our projection data tidy'd correctly, we can now start to generate our final geojson files that we'll need for our d3.js animations and visualizations. In order to merge the data correctly, we'll have to match the `id` column with the `country` column between the geojson property and the dataframe, and then create additional properties to append to each country object. We can then splice the json files into eight separate jsons similar to the process we did for our pandas dataframes. I created the `createjsons()` function below to account for this process.

In [17]:
dataframes = [first20_med_a2, first20_med_b1, second20_med_a2, second20_med_b1, third20_med_a2, third20_med_b1, fourth20_med_a2, fourth20_med_b1]
json_files = ['first20_med_a2.json', 'first20_med_b1.json', 'second20_med_a2.json', 'second20_med_b1.json', 'third20_med_a2.json', 'third20_med_b1.json', 'fourth20_med_a2.json', 'fourth20_med_b1.json']


def createjsons():
    for idx, d in enumerate(dataframes):
        for i in features:
            for index, row in d.iterrows():
                if(row['Country'] == i['id']):
                    i['Change_c'] = row['Change_c']
                    i['Change_f'] = row['Change_f']
                    i['Date_range'] = row['Date_range']
                    i['Percentile'] = row['Pctile']
                    i['Scenario'] = row['Type']
                else:
                    pass
            with open(json_files[idx], 'w') as outfile:
                json.dump(gj, outfile)
    

In [20]:
createjsons()

With the jsons created, we can now upload these into the Angular application and read them into d3.js to build out our visualizations!